## Will a customer leave the bank or stay? An Artificial Neural Networks model
One of the goals of a bank is to expand business by increasing the number of customers. In fact, every business entity has similar goal. Whether a customer is likely stay or leave the bank is a very vital information for a bank from the perspective of business strategy. It is a bank's best interest to figure out in advance the probability of a customer's leaving the bank. If the bank knows in advance that a customer is going to leave, the bank can extend alternative offers for the customer to stay. The model developed in this analysis serves this purpose. Specifically, I build and validate an Artificial Neural Networks (ANN) model to predicts whether any individual customer will leave the bank or stay. Besides, the model also ranks all the customers of the bank, based on their probability of leaving. The data set used to build and validate the model includes information such as customer id, credit score, gender, age, tenure, balance, if the customer is active, has a credit card, etc. as well as a variable that indicates whether the customer left the bank or not.


### Data importing and preprocessing

In [1]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Importing the dataset
dataset = pd.read_csv('Churn_Modelling.csv')

In [2]:
print(dataset.head(2))

   RowNumber  CustomerId   Surname  CreditScore Geography  Gender  Age  \
0          1    15634602  Hargrave          619    France  Female   42   
1          2    15647311      Hill          608     Spain  Female   41   

   Tenure   Balance  NumOfProducts  HasCrCard  IsActiveMember  \
0       2      0.00              1          1               1   
1       1  83807.86              1          0               1   

   EstimatedSalary  Exited  
0        101348.88       1  
1        112542.58       0  


The first three variables are not supposed to have any explanatory power in explaning the dependent variable. Therefore, I exclude these three variables from the dataset.

In [3]:
dataset=dataset.iloc[:, 3:]

In [4]:
print(dataset.head(2))

   CreditScore Geography  Gender  Age  Tenure   Balance  NumOfProducts  \
0          619    France  Female   42       2      0.00              1   
1          608     Spain  Female   41       1  83807.86              1   

   HasCrCard  IsActiveMember  EstimatedSalary  Exited  
0          1               1        101348.88       1  
1          0               1        112542.58       0  


The dataset conatins two categorical variables: 'Geography' and 'Gender'. The following codes converts these categorical variables into numeric variables by creating dummies for each category.Then I exclude one dummy from each dummy variable sets to get rid of the dummy variable trap.

In [5]:
dummy_geo = pd.get_dummies(dataset["Geography"], prefix="Geo")
dummy_sex = pd.get_dummies(dataset["Gender"], prefix="Sex")

In [6]:
# Combine the dummy variables with the original dataset
dataset = pd.concat([dummy_geo.iloc[:, [0,1]], dummy_sex.iloc[:, 0], dataset.iloc[:, [0, 3, 4, 5, 6, 7, 8, 9, 10]]], axis=1)

In [7]:
print(dataset.head(2))

   Geo_France  Geo_Germany  Sex_Female  CreditScore  Age  Tenure   Balance  \
0           1            0           1          619   42       2      0.00   
1           0            0           1          608   41       1  83807.86   

   NumOfProducts  HasCrCard  IsActiveMember  EstimatedSalary  Exited  
0              1          1               1        101348.88       1  
1              1          0               1        112542.58       0  


In [8]:
# Seperate dependent variable and features
X = dataset.iloc[:, 0:len(dataset.columns)-1].values
y = dataset.iloc[:, len(dataset.columns)-1].values

In [9]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [10]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [11]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [12]:
# Importing the Keras libraries and packages
import keras
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [13]:
# Initialising the ANN
classifier = Sequential()
# Adding the input layer and the first hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))

# Adding the second hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))

# Adding the output layer
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

# Compiling the ANN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Fitting the ANN to the Training set
classifier.fit(X_train, y_train, batch_size = 10, epochs = 100)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/100
8000/8000 [==============================] - 2s 207us/step - loss: 0.4868 - acc: 0.7959
Epoch 2/100
8000/8000 [==============================] - 1s 123us/step - loss: 0.4303 - acc: 0.7960
Epoch 3/100
8000/8000 [==============================] - 1s 137us/step - loss: 0.4264 - acc: 0.7960
Epoch 4/100
8000/8000 [==============================] - 1s 133us/step - loss: 0.4217 - acc: 0.8090
Epoch 5/100
8000/8000 [==============================] - 1s 126us/step - loss: 0.4191 - acc: 0.8222
Epoch 6/100
8000/8000 [==============================] - 1s 160us/step - loss: 0.4165 - acc: 0.8256
Epoch 7/100
8000/8000 [==============================] - 1s 131us/step - loss: 0.4149 - acc: 0.8304
Epoch 8/100
8000/8000 [==============================] - 1s 125us/step - loss: 0.4129 - acc: 0.8302
Epoch 9/100
8000/8000 [==============================] - 1s 131us/step - loss: 0

### Making predictions and evaluating the model

In [14]:
# Predicting the Test set results
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1531   64]
 [ 253  152]]


In [15]:
# Find the accuracy rate
from sklearn.metrics import accuracy_score
print("Out of sample accuracy is", accuracy_score(y_test, y_pred, normalize=True))

Out of sample accuracy is 0.8415


Using Artificial Neural Networks model with two hidden layer and without any parameter tuning, the out of sample accuracy for the fitted model is 84.2%. This accuracy is fairly large to validate the model and implement to identify those customers who are more likely to leave the bank. Based on the predicted classification for each customer, the bank can take actions to increase the likihood of retention.